In [1]:
import os
import datetime
import geopandas as gpd
import pandas as pd
import json
import shapely.ops

import sentinelhub
import boto3

/Users/nikhilsrajan/NASA-Harvest/project/fetch_satdata/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import sys
sys.path.append('..')

In [3]:
import config

import cdseutils.utils as utils
import cdseutils.mydataclasses as mydataclasses
import cdseutils.constants as constants
import cdseutils.sentinel2 as sentinel2

import mysecrets
import s2l1c_via_s3

In [4]:
cdse_creds = mysecrets.PROFILES['nikhilsasirajan@gmail.com']

In [5]:
year = 2021

startdate = datetime.datetime(year, 3, 1)
enddate = datetime.datetime(year + 1, 1, 31)

In [6]:
(enddate - startdate).days

336

In [7]:
catalog_filepath = '../data/eth_bb_east_s2l1c_catalog.geojson'
roi_filepath = '../../ethiopia/data/Ethiopia_Bounding_Boxs/Ethiopia_2021_East_Bounding_Box.shp'
# roi_filepath = '../data/test_eth_east_bb_s2grids.geojson',

In [8]:
catalog_gdf = gpd.read_file(catalog_filepath)

In [9]:
catalog_gdf[:2]

,id,timestamp,s3url,cloud_cover,geometry
0,S2A_MSIL1C_20210128T074201_N0500_R092_T37NDJ_2...,2021-01-28 07:57:53.635000+00:00,s3://EODATA/Sentinel-2/MSI/L1C_N0500/2021/01/2...,0.0,"MULTIPOLYGON (((38.09192 8.14101, 39.08859 8.1..."
1,S2A_MSIL1C_20210128T074201_N0500_R092_T37NEJ_2...,2021-01-28 07:57:50.494000+00:00,s3://EODATA/Sentinel-2/MSI/L1C_N0500/2021/01/2...,0.0,"MULTIPOLYGON (((38.99983 8.14203, 39.99648 8.1..."


In [10]:
successful_download_count, total_download_count \
= s2l1c_via_s3.download_sentinel2_l1c_tiles(
    cdse_creds = cdse_creds,
    catalog_gdf = catalog_gdf[:2],
    sentinel2_local_catalog_filepath = config.FILEPATH_SENTINEL2_LOCAL_CATALOG,
    catalog_save_folderpath = config.FOLDERPATH_FETCH_CATALOG_CACHE,
    root_download_folderpath = config.FOLDERPATH_SATELLITE,
    roi_filepath = roi_filepath,
    startdate = startdate,
    enddate = enddate,
    upper_limit_for_number_of_tiles = 50,
    chunksize_for_download_and_update_catalog = 5,
)

100%|██████████| 2/2 [00:01<00:00,  2.00it/s]


100%|██████████| 5/5 [00:25<00:00,  5.00s/it]

Successful downloads: 5 / 5



100%|██████████| 5/5 [00:18<00:00,  3.71s/it]

Successful downloads: 5 / 5


...

Successful downloads: 5 / 5



100%|██████████| 5/5 [00:00<00:00,  7.05it/s]

Successful downloads: 5 / 5



100%|██████████| 1/1 [00:00<00:00,  2.30it/s]

Successful downloads: 1 / 1
Successful downloads: 26 / 26


In [11]:
catalog_gdf = gpd.read_file(config.FILEPATH_SENTINEL2_LOCAL_CATALOG)

In [12]:
catalog_gdf

,id,timestamp,s3url,local_folderpath,files,last_update,cloud_cover,geometry
0,S2A_MSIL1C_20210128T074201_N0500_R092_T37NDJ_2...,2021-01-28 07:57:53.635000+00:00,s3://EODATA/Sentinel-2/MSI/L1C_N0500/2021/01/2...,../data/satellite/Sentinel-2/MSI/L1C_N0500/202...,"B8A.jp2,B12.jp2,B10.jp2,B01.jp2,B06.jp2,B03.jp...",2024-08-20 17:37:41.223000+00:00,0.0,"MULTIPOLYGON (((38.09192 8.14101, 39.08859 8.1..."
1,S2A_MSIL1C_20210128T074201_N0500_R092_T37NEJ_2...,2021-01-28 07:57:50.494000+00:00,s3://EODATA/Sentinel-2/MSI/L1C_N0500/2021/01/2...,../data/satellite/Sentinel-2/MSI/L1C_N0500/202...,"B8A.jp2,B12.jp2,B10.jp2,B06.jp2,B01.jp2,B03.jp...",2024-08-20 17:37:52.890000+00:00,0.0,"MULTIPOLYGON (((38.99983 8.14203, 39.99648 8.1..."
